# Notebook: 01 - Data Cleaning & Preprocessing

**1. Import Required Libraries**

In [90]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer, SimpleImputer

**2. Define Paths and Ensure Directory Structure**

In [ ]:
# Defining the base directory
base_dir = os.path.dirname(os.getcwd())

In [ ]:
# Constructing the file paths
ieee_trans_path = os.path.join(base_dir, 'data', 'raw', 'train_transaction.csv')
ieee_id_path = os.path.join(base_dir, 'data', 'raw', 'train_identity.csv')
credit_df_path = os.path.join(base_dir, 'data', 'raw', 'creditcard.csv')
processed_dir = os.path.join(base_dir, 'data', 'processed')


In [110]:
# Ensure the processed directory exists
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

**3. Load Datasets**

In [93]:
# Loading the Credit Card Fraud Detection dataset
credit_df = pd.read_csv(credit_df_path)

In [94]:
# Loading the IEEE-CIS Fraud Detection dataset
ieee_trans = pd.read_csv(ieee_trans_path)
ieee_id = pd.read_csv(ieee_id_path)

**4. Display Basic Information of the Datasets**

In [95]:
# Displaying the first five rows of the Credit Card Fraud Detection dataset
print(credit_df.head())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [96]:
# Displaying the first five rows of the IEEE-CIS Fraud Detection dataset
print(ieee_trans.head())
print(ieee_id.head())

   TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5  ... V330  V331  V332  V333  V334 V335  \
0    NaN  150.0    discover  142.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
1  404.0  150.0  mastercard  102.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
2  490.0  150.0        visa  166.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
3  567.0  150.0  mastercard  117.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
4  514.0  150.0  mastercard  102.0  ...  0.0   0.0   0.0   0.0   0.0  0.0   

  V336  V337  V338  V339  
0  NaN   NaN   NaN   NaN  
1  NaN   NaN   NaN  

In [97]:
# Displaiying the information of the Credit Card Fraud Detection dataset
print(credit_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

- For Credit Card Fraud Dataset, all columns are numerical due to PCA transformation (V1-V28).
- Amount is not normalized, and Class is binary (0 = Non-Fraud, 1 = Fraud).
- For IEEE-CIS, we see a mix of numerical and categorical variables, meaning encoding will be required.
- There are NaN values, indicating missing data.

In [98]:
# Displaiying the information of the IEEE-CIS Fraud Detection dataset
print(ieee_trans.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB
None


In [99]:
print(ieee_id.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  144233 non-null  int64  
 1   id_01          144233 non-null  float64
 2   id_02          140872 non-null  float64
 3   id_03          66324 non-null   float64
 4   id_04          66324 non-null   float64
 5   id_05          136865 non-null  float64
 6   id_06          136865 non-null  float64
 7   id_07          5155 non-null    float64
 8   id_08          5155 non-null    float64
 9   id_09          74926 non-null   float64
 10  id_10          74926 non-null   float64
 11  id_11          140978 non-null  float64
 12  id_12          144233 non-null  object 
 13  id_13          127320 non-null  float64
 14  id_14          80044 non-null   float64
 15  id_15          140985 non-null  object 
 16  id_16          129340 non-null  object 
 17  id_17          139369 non-nul

**5. Merging IEEE Transaction and Identity Data**

In [ ]:
ieee_df = ieee_trans.merge(ieee_id, on="TransactionID", how="left")

**6. Handling Missing Values**

In [ ]:
# Drop columns with more than 50% missing values
threshold = 0.5
ieee_df = ieee_df.dropna(thresh=len(ieee_df) * (1 - threshold), axis=1)

In [102]:
# Imputing missing values
num_cols = ieee_df.select_dtypes(include=['float64', 'int64']).columns
num_imputer = SimpleImputer(strategy='median')
ieee_df[num_cols] = num_imputer.fit_transform(ieee_df[num_cols])

In [103]:
# Filling categorical columns with 'unknown'
cat_cols = ieee_df.select_dtypes(include=['object']).columns
ieee_df[cat_cols] = ieee_df[cat_cols].fillna("unknown")

**7. Scaling Numerical Features**

In [ ]:
# Standardizing the 'Amount' column for the Credit Card Fraud Detection dataset
scaler = StandardScaler()
credit_df['Amount'] = scaler.fit_transform(credit_df[['Amount']])


**8. Encoding Categorical Variables**

In [105]:
categorical_cols = ieee_df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    ieee_df[col] = ieee_df[col].fillna('unknown')
    ieee_df[col] = LabelEncoder().fit_transform(ieee_df[col])

**9. Balancing the Datasets using SMOTE**

In [106]:
# Balancing the Credit Card Fraud Detection dataset
X_credit = credit_df.drop('Class', axis=1)
y_credit = credit_df['Class']
smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_credit_res, y_credit_res = smote.fit_resample(X_credit, y_credit)
credit_df_balanced = pd.concat([X_credit_res, y_credit_res], axis=1)

In [ ]:
# Balancing the IEEE-CIS Fraud Detection dataset
X_ieee = ieee_df.drop('isFraud', axis=1)
y_ieee = ieee_df['isFraud']
X_ieee_resampled, y_ieee_resampled = smote.fit_resample(X_ieee, y_ieee)
ieee_df_resampled = pd.concat([X_ieee_resampled, y_ieee_resampled], axis=1)


**10. Saving the Processed Data**

In [ ]:
# Saving the processed data
credit_df_resampled = pd.concat([X_credit, y_credit], axis=1)
credit_df_resampled.to_csv(os.path.join(processed_dir, "credit_cleaned.csv"), index=False)
ieee_df_resampled.to_csv(os.path.join(processed_dir, "ieee_cleaned.csv"), index=False)


In [ ]:
# Quick check of preprocessed data
print("Credit dataset after preprocessing:")
credit_df_resampled.head()

Credit dataset after preprocessing:


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [ ]:
print("IEEE dataset after preprocessing:")
ieee_df_resampled.head()

IEEE dataset after preprocessing:


,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V313,V314,V315,V316,V317,V318,V319,V320,V321,isFraud
0,2987000.0,86400.0,68.5,4,13926.0,361.0,150.0,1,142.0,1,...,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0
1,2987001.0,86401.0,29.0,4,2755.0,404.0,150.0,2,102.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987002.0,86469.0,59.0,4,4663.0,490.0,150.0,4,166.0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987003.0,86499.0,50.0,4,18132.0,567.0,150.0,2,117.0,2,...,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,0.0
4,2987004.0,86506.0,50.0,1,4497.0,514.0,150.0,2,102.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
